In [1]:
# !pip install imblearn

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
df = pd.DataFrame(pd.read_csv('files/playground-series-s4e6/train.csv'))



In [5]:
categorical_features = ['Marital status' ,'Application mode' ,'Application order' ,'Course'
                        ,'Previous qualification' ,'Nacionality' ,"Mother's qualification"
                        ,"Father's qualification" ,"Mother's occupation" ,"Father's occupation"]


In [6]:
categorical_features = ['Marital status' ,'Application mode' ,'Application order' ,'Course'
                        ,'Previous qualification' ,'Nacionality']

In [7]:
colums = [ 'Marital status' ,'Application mode' ,'Application order' ,'Course'
        ,'Previous qualification' ,'Nacionality', 'Debtor',
        'Tuition fees up to date', 'Gender', 'Scholarship holder',
        'Age at enrollment', 'International','Inflation rate', 'GDP', 'Target']

In [8]:
df = df[colums]

In [9]:
#categorical_features = df.drop('Target',axis=1).columns
for var in categorical_features:
    cat_list='var'+'_'+ var
    cat_list = pd.get_dummies(df[var], prefix=var)
    data1=df.join(cat_list)
    df=data1
    df.drop(var, axis=1, inplace=True)

df.head()

,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Inflation rate,GDP,Target,Marital status_1,...,Nacionality_25,Nacionality_26,Nacionality_32,Nacionality_41,Nacionality_62,Nacionality_100,Nacionality_101,Nacionality_103,Nacionality_105,Nacionality_109
0,0,1,0,1,18,0,0.6,2.02,Graduate,True,...,False,False,False,False,False,False,False,False,False,False
1,0,1,0,0,18,0,0.6,2.02,Dropout,True,...,False,False,False,False,False,False,False,False,False,False
2,0,1,1,0,18,0,0.3,-0.92,Dropout,True,...,False,False,False,False,False,False,False,False,False,False
3,0,1,0,1,18,0,0.6,2.02,Enrolled,True,...,False,False,False,False,False,False,False,False,False,False
4,0,1,0,0,18,0,2.6,0.32,Graduate,True,...,False,False,False,False,False,False,False,False,False,False


In [10]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_info_rows = 1000*7
pd.options.display.float_format = '{:.6f}'.format

In [11]:
df.head()

,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Inflation rate,GDP,Target,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_2,Application mode_3,Application mode_4,Application mode_5,Application mode_7,Application mode_9,Application mode_10,Application mode_12,Application mode_15,Application mode_16,Application mode_17,Application mode_18,Application mode_26,Application mode_27,Application mode_35,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_51,Application mode_53,Application order_0,Application order_1,Application order_2,Application order_3,Application order_4,Application order_5,Application order_6,Application order_9,Course_33,Course_39,Course_171,Course_979,Course_8014,Course_9003,Course_9070,Course_9085,Course_9119,Course_9130,Course_9147,Course_9238,Course_9254,Course_9500,Course_9556,Course_9670,Course_9773,Course_9853,Course_9991,Previous qualification_1,Previous qualification_2,Previous qualification_3,Previous qualification_4,Previous qualification_5,Previous qualification_6,Previous qualification_9,Previous qualification_10,Previous qualification_11,Previous qualification_12,Previous qualification_14,Previous qualification_15,Previous qualification_17,Previous qualification_19,Previous qualification_36,Previous qualification_37,Previous qualification_38,Previous qualification_39,Previous qualification_40,Previous qualification_42,Previous qualification_43,Nacionality_1,Nacionality_2,Nacionality_6,Nacionality_11,Nacionality_17,Nacionality_21,Nacionality_22,Nacionality_24,Nacionality_25,Nacionality_26,Nacionality_32,Nacionality_41,Nacionality_62,Nacionality_100,Nacionality_101,Nacionality_103,Nacionality_105,Nacionality_109
0,0,1,0,1,18,0,0.600000,2.020000,Graduate,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,1,0,0,18,0,0.600000,2.020000,Dropout,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,1,1,0,18,0,0.300000,-0.920000,Dropout,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,1,0,1,18,0,0.600000,2.020000,Enrolled,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,F

In [12]:
# Criar um dicionário de mapeamento
target_map = {
    'Graduate': 1,
    'Dropout': 2,
    'Enrolled': 3
}

df['Target'] = df['Target'].map(target_map)

In [13]:
categorical = df.drop('Target',axis=1)
target = df['Target']

In [14]:
X = categorical.copy()
y = target.copy()

In [15]:
X.head()

,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Inflation rate,GDP,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_2,Application mode_3,Application mode_4,Application mode_5,Application mode_7,Application mode_9,Application mode_10,Application mode_12,Application mode_15,Application mode_16,Application mode_17,Application mode_18,Application mode_26,Application mode_27,Application mode_35,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_51,Application mode_53,Application order_0,Application order_1,Application order_2,Application order_3,Application order_4,Application order_5,Application order_6,Application order_9,Course_33,Course_39,Course_171,Course_979,Course_8014,Course_9003,Course_9070,Course_9085,Course_9119,Course_9130,Course_9147,Course_9238,Course_9254,Course_9500,Course_9556,Course_9670,Course_9773,Course_9853,Course_9991,Previous qualification_1,Previous qualification_2,Previous qualification_3,Previous qualification_4,Previous qualification_5,Previous qualification_6,Previous qualification_9,Previous qualification_10,Previous qualification_11,Previous qualification_12,Previous qualification_14,Previous qualification_15,Previous qualification_17,Previous qualification_19,Previous qualification_36,Previous qualification_37,Previous qualification_38,Previous qualification_39,Previous qualification_40,Previous qualification_42,Previous qualification_43,Nacionality_1,Nacionality_2,Nacionality_6,Nacionality_11,Nacionality_17,Nacionality_21,Nacionality_22,Nacionality_24,Nacionality_25,Nacionality_26,Nacionality_32,Nacionality_41,Nacionality_62,Nacionality_100,Nacionality_101,Nacionality_103,Nacionality_105,Nacionality_109
0,0,1,0,1,18,0,0.600000,2.020000,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,1,0,0,18,0,0.600000,2.020000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,1,1,0,18,0,0.300000,-0.920000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,1,0,1,18,0,0.600000,2.020000,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,F

In [16]:
y.head()

0    1
1    2
2    2
3    3
4    1
Name: Target, dtype: int64

In [17]:

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Amostragem
smote = SMOTE()
X, y = smote.fit_resample(X, y)

# Adicionar características polinomiais
poly = PolynomialFeatures(degree=2)
X = poly.fit_transform(X)

# Ajuste de hiperparâmetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}


In [18]:
# pd.DataFrame(X).head()

In [19]:
# pd.DataFrame(y).head()

: 

In [20]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:

grid = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)


In [ ]:
grid.best_estimator_

In [ ]:
# Treinar o modelo com os melhores parâmetros
model = grid.best_estimator_
model.fit(X_train, y_train)


# Avaliação
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(f"Melhor acurácia: {grid.best_score_}")


In [ ]:
import joblib

joblib.dump(model, 'model_logistic.joblib')

In [ ]:
# Carregar o modelo salvo
model = joblib.load('model_logistic.joblib')

# Exemplo de novos dados para previsão
df_test = pd.read_csv('files/playground-series-s4e6/test.csv')

df_test.head()

In [ ]:

# Fazer previsões com o modelo carregado
predict = model.predict(df_test)

predict